In [1]:
import pandas as pd
from unidecode import unidecode

In [ ]:
def carregar_e_mesclar_arquivos(url_exp_2024: str, url_pais: str, 
                                chave_principal: str, chave_referencia: str, 
                                sep_principal: str = ',', sep_referencia: str = ';', 
                                encoding: str = 'latin1'):
    
    df_exp_2024 = pd.read_csv(url_exp_2024, sep=sep_principal, encoding=encoding)
    df_pais = pd.read_csv(url_pais, sep=sep_referencia, encoding=encoding)

    df_exp_2024[chave_principal] = df_exp_2024[chave_principal].astype(str)
    df_pais[chave_referencia] = df_pais[chave_referencia].astype(str)

    df_resultado = df_exp_2024.merge(df_pais, left_on=chave_principal, right_on=chave_referencia, how='left')

    return df_resultado

df_1 = carregar_e_mesclar_arquivos(
    url_exp_2024='../../../../datalake/landing/exportacao_2024.csv',
    url_pais='../../../../datalake/raw_data/PAIS.csv',
    chave_principal='cd_pais',
    chave_referencia='CO_PAIS'
)

In [3]:
df_1 = df_1.drop(columns=['cd_pais', 'CO_PAIS', 'CO_PAIS_ISON3', 
        'CO_PAIS_ISOA3', 'NO_PAIS_ING', 'NO_PAIS_ESP'])

In [ ]:
url_urf = '../../../../datalake/raw_data/URF.csv'
df_urf = pd.read_csv(url_urf, sep=';',  encoding='latin1')

In [5]:
def mesclar_dataframes(df_base: pd.DataFrame, df_referencia: pd.DataFrame, 
                       chave_base: str, chave_referencia: str) -> pd.DataFrame:

    df_base[chave_base] = df_base[chave_base].astype(str)
    df_referencia[chave_referencia] = df_referencia[chave_referencia].astype(str)

    df_resultante = df_base.merge(df_referencia, left_on=chave_base, right_on=chave_referencia, how='left')
    
    return df_resultante

df_2 = mesclar_dataframes(df_base=df_1, df_referencia=df_urf, chave_base='cd_urf', chave_referencia='CO_URF')


In [6]:
df_2['urf_codigo'] = df_2['NO_URF'].str.extract(r'^(\d+)\s*-\s*')

df_2['urf_info'] = df_2['NO_URF'].str.replace(r'^\d+\s*-\s*', '', regex=True)

def separar_info(valor):
    partes = valor.split(' - ')
    if len(partes) == 2:
        tipo, nome = partes
    else:
        tipo = None
        nome = valor
    return pd.Series([tipo, nome])

df_2[['urf_tipo', 'urf_nome']] = df_2['urf_info'].apply(separar_info)

df_2.drop(columns=['NO_URF', 'urf_info'], inplace=True)

In [7]:
df_2 = df_2.drop(['cd_urf', 'CO_URF', 'urf_codigo', 'urf_tipo'], axis=1)

In [ ]:
url_ncm = '../../../../datalake/raw_data/NCM.csv'
df_ncm = pd.read_csv(url_ncm, sep=';',  encoding='latin1')

In [9]:
def mesclar_dataframes(df_base: pd.DataFrame, df_referencia: pd.DataFrame, 
                       chave_base: str, chave_referencia: str) -> pd.DataFrame:
    
    df_base[chave_base] = df_base[chave_base].astype(str)
    df_referencia[chave_referencia] = df_referencia[chave_referencia].astype(str)

    df_resultante = df_base.merge(df_referencia, left_on=chave_base, right_on=chave_referencia, how='left')
    
    return df_resultante

df_3 = mesclar_dataframes(df_base=df_2, df_referencia=df_ncm, chave_base='cd_ncm', chave_referencia='CO_NCM')

In [10]:
df_3 = df_3.drop(columns=['cd_ncm', 
       'CO_NCM', 'CO_UNID', 'CO_SH6', 'CO_PPE', 'CO_PPI', 'CO_FAT_AGREG',
       'CO_CUCI_ITEM', 'CO_CGCE_N3', 'CO_SIIT', 'CO_ISIC_CLASSE',
       'CO_EXP_SUBSET', 'NO_NCM_ESP', 'NO_NCM_ING'])

In [ ]:
url_ncm_unidade = '../../../../datalake/raw_data/NCM_UNIDADE.csv'
df_ncm_unidade = pd.read_csv(url_ncm_unidade, sep=';',  encoding='latin1') 


In [12]:
def mesclar_dataframes(df_base: pd.DataFrame, df_referencia: pd.DataFrame, 
                       chave_base: str, chave_referencia: str) -> pd.DataFrame:
    
    df_base[chave_base] = df_base[chave_base].astype(str)
    df_referencia[chave_referencia] = df_referencia[chave_referencia].astype(str)

    df_resultante = df_base.merge(df_referencia, left_on=chave_base, right_on=chave_referencia, how='left')
    
    return df_resultante

df_4 = mesclar_dataframes(df_base=df_3, df_referencia=df_ncm_unidade, chave_base='cd_unidade', chave_referencia='CO_UNID')

In [13]:
df_4 = df_4.drop(columns=['cd_unidade',
       'CO_UNID', 'SG_UNID'])

In [ ]:
url_via = '../../../../datalake/raw_data/VIA.csv' 
df_via = pd.read_csv(url_via, sep=';',  encoding='latin1')  

In [15]:
def mesclar_dataframes(df_base: pd.DataFrame, df_referencia: pd.DataFrame, 
                       chave_base: str, chave_referencia: str) -> pd.DataFrame:

    df_base[chave_base] = df_base[chave_base].astype(str)
    df_referencia[chave_referencia] = df_referencia[chave_referencia].astype(str)

    df_resultante = df_base.merge(df_referencia, left_on=chave_base, right_on=chave_referencia, how='left')
    
    return df_resultante

df_5 = mesclar_dataframes(df_base=df_4, df_referencia=df_via, chave_base='cd_via', chave_referencia='CO_VIA')

In [16]:
df_5 = df_5.drop(columns=['cd_via', 'CO_VIA'])

In [17]:
df_5 = df_5.rename(columns={  
  'sg_ufncm': 'sg_uf',
  'NO_PAIS': 'cd_pais', 
  'urf_nome': 'cd_urf', 
  'NO_NCM_POR': 'cd_ncm',
  'NO_UNID': 'cd_unidade',
  'NO_VIA': 'cd_via'
    
    })
df_5.columns = df_5.columns.str.strip().str.lower().str.replace(' ', '_')

In [18]:
df_5.head()

,sg_uf,qt_estatistica,kg_liquido,vl_dolarfob,data_exportacao,cd_pais,cd_urf,cd_ncm,cd_unidade,cd_via
0,sp,8,425,20423,2024-04-01,Espanha,AEROPORTO INTERNACIONAL DE SAO PAULO/GUARULHOS,Outros móveis de madeira,NUMERO (UNIDADE),AEREA
1,sp,418615,60355,470512,2024-11-01,México,PORTO DE SANTOS,Amortecedores de suspensão para tratores e veí...,NUMERO (UNIDADE),MARITIMA
2,sp,94080,94080,237562,2024-03-01,Comores,PORTO DE SANTOS,"Pedaços e miudezas, comestíveis de galos/galin...",QUILOGRAMA LIQUIDO,MARITIMA
3,mg,10,12,1704,2024-04-01,Chile,PORTO DE SANTOS,Outros aparelhos e instrumentos sem dispositiv...,NUMERO (UNIDADE),MARITIMA
4,sc,26,26,950,2024-08-01,Chile,AEROPORTO INTERNACIONAL DE VIRACOPOS,"Protetores, bandas de rodagem, etc, para pneus...",QUILOGRAMA LIQUIDO,AEREA


In [18]:
def padronizar_colunas(df, colunas):
    df[colunas] = df[colunas].apply(lambda x: x.str.strip().str.lower())
    return df

df_5 = padronizar_colunas(df_5, ['cd_pais', 'cd_urf', 'cd_ncm', 'cd_unidade', 'cd_via'])

In [19]:
for col in df_5.select_dtypes(include='object').columns:
    df_5[col] = df_5[col].apply(lambda x: unidecode(x) if isinstance(x, str) else x)

In [ ]:
caminho = "../../../../datalake/silver/exportacao_2024_m.csv" #salvando o arquivo
df_5.to_csv(caminho, index=False)